### Цель проекта: построить модель, которая сможет предсказывать зарплату по текстовому описанию вакансии.

Сам датасет взят из соревнования "2nd step in NLP", которое недавно поводила ВШЭ.
В бейзлайне который был предоставлен организаторами использовался Gensim'овский НЕ-предобученный Word2Vec, который обучили на колонке с описаниями вакансий, получив эмбеддинги для каждого слова и усреднив их, тем самым получив W2V всего описания вакансии. Далее для предсказания зарплаты использовали обычную линейную регрессию с L1 регуляризатором. 

Я попробую нсколько подходов, состоящих из комбинации способа получения вектора текста и предсказательной модели:
Получение векторов:
- различные вариации TF-IDF
- Word2Vec

Предсказательная модель:
- Регресия с различными вариантами регуляризации
- Градиентный бустинг

#### Описание датасета: 
Датасет сождержит 16231 уникальных строк с данными по вакансиям, а именно:
- название вакансии
- режим работы
- тип занятости
- описание вакансии
- требуемые навыки
- зарплата



In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=True)


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [12, 8]

import re
from wordcloud import WordCloud
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from tqdm import tqdm

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDRegressor
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import scipy
from scipy.sparse import hstack

import catboost
from catboost import CatBoostRegressor
import optuna

from gensim.models import Word2Vec
import nltk

import random
random.seed(1337)
np.random.seed(1337)
rng = 1337

In [ ]:
df = pd.read_csv('vacancies_train.csv')

In [ ]:
df.head(3)

In [ ]:
df.isna().sum()

In [ ]:
df.info()

Пропусков нет, кроме колонки "требуемые навыки", но она нас не так сильно волнует.

In [ ]:
# посмотрим есть ли задвоения строк:
df[df.duplicated(keep=False)].sort_values(by='salary')

In [ ]:
# удалим дубликаты оставив первые вхождения, и посмотрим сколько осталось:
df.drop_duplicates(inplace=True, ignore_index=True)
df.shape

### EDA and Text Preprocessing:

Перед тем как переходить к обработке текста, посмотрим что вообще из себя представляют данные, для начала визуально посмотрим какие профессии больше всего ищут:

In [ ]:
stop_words = set(stopwords.words('russian'))

wordcloud = WordCloud(
    stopwords=stop_words, background_color='white', width=1400, height=800).generate_from_frequencies(df.name.value_counts())
plt.figure(figsize=(15,12))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.show()

Бегло посмотрим на рпаспределение типов занятостии графика работы:

In [ ]:
plt.figure(figsize=(7,4))
plt.xticks(rotation=25)
sns.histplot(df['schedule'], color='#DAF7A6', shrink=0.5);

In [ ]:
plt.figure(figsize=(7,4))
plt.xticks(rotation=25)
sns.histplot(df['employment'], color='#DAF7A6', shrink=0.5);

Вполне ожидаемо, абсолютно большая часть - полный день + полная занятость. \
Посмотрим на распределение целевой переменной - зарплаты:

In [ ]:
df['salary'].describe()

In [ ]:
plt.hist(df.salary, bins = 100, rwidth=0.5);

Похоже на лог-нормальное распределение, попробуем отлогарифмировать:

In [ ]:
plt.hist(np.log(df.salary), bins = 100, rwidth=0.5);

А что там с совместным распределением зарплат по типу занятости опыту?

In [ ]:
# cut=0 чтобы на графике отображались только днаные из датасета, без сглаживания,
# которое может визуально показать отрицательную зарплату.
# scale='area' чтобы нивелировать разницу в частоте видов занятости.
sns.violinplot(data=df, y=df['salary']/1000, x='schedule', hue='employment',
               scale='area', cut=0) 
plt.ylabel('salary (1,000s)')
plt.ylim(-10, 250); # отрезаем зарплты 250+, поскольку они являются выбросами,
                   # и визуально только помешают оценить взаимсвязи

Четкую зависимость установить трудно, разве что вахтовый метод выбивается в большую сторону, что вполне логично, перекодировать в числительный не будем

In [ ]:
sns.violinplot(data=df, y=df['salary']/1000, x='experience', hue='schedule',
               order=df.experience.value_counts().keys(), scale='area', cut=0) 
plt.ylabel('salary (1,000s)')
plt.ylim(-10, 250);

Тут пару выводов выводов: \
1 - четко виден тренд, что зарплата растет в зависимости от требуемого опыта, поэтому будем перекодировать признак опыта на числительный 1-4, где 1 - "нет опыта", а 4 - "более 6 лет". \
2 - огромные выбросы в большую сторону, даже там где опыт не требуется, подозреваю что это вакансии связанные с продажами, с низким окладом, и "безграничными возможностями заработать миллиарды" как % от продаж. И небольшие (логично ограниченные нулем) выбросы в меньшую сторону.

#### Я не хочу удалять выбросы в большую сторону, так как это приведет к потере части информации, выбросы в меньшую сторону обусловлены почасовой или посменной оплатой, хочется исправить, но это крайне проблемматично, поскольку потребует ручных правок, если их не очень много, то оставлю как есть

In [ ]:
df[df.salary < 10000].shape

59 вакансий с зарплатой меньше 10 тыс, выборосы в меньшую оставляем как есть.
___

#### Предобработка:
Перед тем как преобразовывать текст в вектора методами вроде TFIDF / Word2Vec, его нужно подготовить, очистить от знаков препинания, лишних символов, стоп слов, привести слова к нормальной форме. \
Для очистки напишем несколько функций:


In [ ]:
# создадим новый датафрейм, где будут очищенные данные:
df_p = pd.DataFrame() # p - processed

# Создадим функцию для замены признака опыта на количественный:
def conditions(x):
    if x == 'Нет опыта': return 1
    elif x == 'От 1 года до 3 лет': return 2
    elif x == 'От 3 до 6 лет': return 3
    else: return 4

conditions_vect = np.vectorize(conditions)

In [ ]:
# мусорные, не несущие особого смысла слова, кандидаты на удаление,
#stop_words = set(stopwords.words('russian'))

print('Примеры стоп слов:')
for i in range(3):
    print(list(stop_words)[i])
print(f'всего стоп слов: {len(stop_words)}')

In [ ]:
# функция для чистки текстов (описаний) от разного регистра, и всякого мусора
# вроде стопслов или не текстовых значений
def initial_processing(corpus):
    n = len(corpus)
    for i in range(n):
        corpus[i] = corpus[i].lower()
        corpus[i] = re.sub(r'[^а-яА-Яa-zA-Z0-9]', ' ', corpus[i])
        corpus[i] = [word for word in corpus[i].split() if word not in stop_words and len(word) > 1]
        corpus[i] = ' '.join(corpus[i])
    return corpus

In [ ]:
# Для примера:
# текст описания ДО:
example_text = df['description'][0:2].copy()
example_text[0]

In [ ]:
# текст описания ПОСЛЕ:
a = initial_processing(example_text)
a[0]

Будем использовать признаки:
 - название вакасии
 - описание вакансии
 - требуемый опыт
 - ключевые навыки

In [ ]:
df_p['name'] = initial_processing(df['name'].copy())
df_p['description'] = initial_processing(df['description'].copy())
df_p['experience'] = conditions_vect(df['experience'])
df_p['key_skills'] = initial_processing(df['key_skills'].copy())

In [ ]:
df_p.head(3)

In [ ]:
morph = MorphAnalyzer()

# Принимаем слово, возвращаем нормальную форму,
# также используем кэширование для ускорения (много одинаковых итераций):
@lru_cache(maxsize=256)
def lemmatize_word(word):
    return morph.parse(word)[0].normal_form 

# Пробегаемся по корпусу, в каждом куске, пробегаемся по всем словам, передавая их в функцию выше
# и сохраняем преобразованные параграфы:
def lemmatize(corpus):
    n = len(corpus)
    with tqdm(total=n) as pbar:
        for i in range(n):
            corpus[i] = ' '.join([lemmatize_word(word) for word in corpus[i].split()])
            pbar.update(1)
    return corpus

In [ ]:
df_p['name'] = lemmatize(df_p['name'].copy())
df_p['description'] = lemmatize(df_p['description'].copy())
df_p['key_skills'] = lemmatize(df_p['key_skills'].copy())

In [ ]:
df_p.head()

Готово, теперь перейдем разделим данные на тренировочные и тестовые, и перейдем к 2 главным частям, векторизации и построению предсказательной модели.

In [ ]:
y = df['salary']
X_train, X_test, y_train, y_test = train_test_split(df_p, y, test_size=0.25, random_state=3)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

#### Небольшой дисклеймер: этапом создания новых признаков буду считать векторизацию, так как это и есть создание признаков, которые пойдут на вход модели предсказания.

### Baseline: TF-IDF векторизация описания вакансии с дефолтными параметрами + линейная регрессия со стохастическим градиентным спуском
Коротко про TF-IDF, это скажем так развитие Bag of Words (BoW). В BoW мы создаем словарь из всех уникальных токенов (слов) во всем нашем тексте (корпусе), и дальше можем каждому отрывку общего текста (предложению, параграфу) присвоить вектор длинны словаря, где каждый элемент - количество раз, сколько конкретное слово из словаря встречается в этом отрывке.\
\
Это самый простой способ превратить текст в цифры для понимания компьютера. Но и самый бесполезный, никак не учитывающий даже важность слов, не говоря уже о порядке слов и семантике. Также, при большом корпусе, словарь раздувается до гигантских размеров, особенно если составлять его не только из еденичных слов, но и н-грамм. А вектора для отрывков будут на 99% состоять из нулей.\
\
Н-граммы это последовательности из слов, например можно рассматривать "Высшая Школа Экономики" как 3 отдельных токена, или объеденить их в одну триграмму. в первом случае, предложения "В школе преподают экономику на высшем уровне" и "В Высшей Школе Экономики кофе делают на уровне" будут иметь довольно схожие вектора, а вот при добавлении н-грамм появится больше различий.\
\
TF-IDF это попытка учесть важность слов, мы все еще создаем общий словарь, и каждому отрывку (предложению) присваиваем вектор длинны словаря, но значения в этом векторе, это уже не просто количество вхождений. Они считаются так: 

$$TF = \frac{сколько\ раз\ слово\ вошло\ в\ отрывок}{сколько\ слов\ во\ всем\ корпусе}$$

$$IDF = log(\frac{сколько\ отрывков\ в\ корпусе}{сколько\ отрывков\ в\ корпусе\ которые\ содержат\ данное\ слово})$$
$$ TF-IDF = TF * IDF $$
Смысл тут в том, что если некое слово часто встречается в отрывке (параграфе), но при этом крайне редко встречается в остальном корпусе, оно видимо несет важную информацию, а вот если оно встречается везде, то видимо оно ничего особо не значит (самые частые слова, например "что, и, от, из" вообзе можно удалить, приняв за "стоп слова", что мы и сделали на этапе предварительной обработки). \
\
BoW в чистом виде я не вижу смысла использовать, сразу возьмем TF-IDF:

In [ ]:
# Для иллюстрации размеров BoW
count = CountVectorizer()
X_count = count.fit(X_train['description'])

count_ngram = CountVectorizer(ngram_range=(1,2))
X_count_ngram = count_ngram.fit(X_train['description'])

print(f'длинна словаря только с юниграммами: {len(X_count.vocabulary_)}')
print(f'длинна словаря с биграммами: {len(X_count_ngram.vocabulary_)}')

In [ ]:
baseline_tfidf = TfidfVectorizer() #только юниграмы
X_train_tfidf = baseline_tfidf.fit_transform(X_train['description'])

In [ ]:
X_train_tfidf

В результате мы получили разряженную матрицу (для экономии памяти, так как большинство значений все равно 0), каждя строка которой - описание вакансии представленное в виде TF-IDF вектора с длинной равной количеству слов в общем словаре (33366)\
\
Теперь построим предсказательную модель:

In [ ]:
%%time
reg = SGDRegressor(max_iter=2000, learning_rate='adaptive', penalty = 'l2', random_state=rng)
reg.fit(X_train_tfidf, y_train) 

In [ ]:
# кол-во итераций подобрано импирически, с адаптивным learning-rate и l2 регуляризатором
# модель достаточно быстро сходится, ниже реальное кол-во итераций пройденных моделью
# запас оставлен для кросс валидации
reg.n_iter_

In [ ]:
X_test_tfidf = baseline_tfidf.transform(X_test['description'])
pred = reg.predict(X_test_tfidf)
baseline_r2 = r2_score(y_test, pred)
baseline_r2

Это очень хороший, в бейзлайне самого соревнования, они получили чтото около 0.23. Первое место заняла модель с 0.51. Сразу же отмечу, что я регрессия с l-2 регуляризацией гораздо лучше себя показывает чем с l1 или вообще без нее. Эти варианты я пробовал, но отбросил сразу. Сделаем кросс-валидацию:

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(reg, X_train_tfidf, y_train,
                         scoring='r2', cv=10, verbose=2, n_jobs=-1)

In [ ]:
baseline_cv = np.mean(scores)
scores, baseline_cv

Очень неплохо для бейзлайна. Посмотрим что можно улучшить.
___
#### Оптимизация:
Я разделю оптимизацию на 2 части, улучшение векторизации, и моделирование, сначала, используя бейзлайновую модель регрессии, попробуем улучшить входные данные, поменяв векторизацию:

In [ ]:
# Попробуем использовать 3 дополнительные колонки: название, ключевые навыки и опыт работы:

vect_desc, vect_name, vect_skills = TfidfVectorizer(), TfidfVectorizer(), TfidfVectorizer()

X_train_desc = vect_desc.fit_transform(X_train['description'])
X_train_name = vect_name.fit_transform(X_train['name'])
X_train_skills = vect_skills.fit_transform(X_train['key_skills'])
X_train_exp = scipy.sparse.csc_matrix(X_train['experience']).transpose()

X_train_tfidf = hstack([X_train_desc, X_train_name, X_train_skills, X_train_exp])

In [ ]:
X_train_tfidf.shape

In [ ]:
reg = SGDRegressor(max_iter=30000, learning_rate='adaptive', penalty = 'l2', random_state=rng)
reg.fit(X_train_tfidf, y_train)

In [ ]:
X_test_desc = vect_desc.transform(X_test['description'])
X_test_name = vect_name.transform(X_test['name'])
X_test_skills = vect_skills.transform(X_test['key_skills'])
X_test_exp = scipy.sparse.csc_matrix(X_test['experience']).transpose()

X_test_tfidf = hstack([X_test_desc, X_test_name, X_test_skills, X_test_exp])

pred = reg.predict(X_test_tfidf)
all_text_r2 = r2_score(y_test, pred)
all_text_r2

Небольшое улучшение, посмотрим что будет на кросс валидации:

In [ ]:
scores = cross_val_score(reg, X_train_tfidf, y_train,
                         scoring='r2', cv=5, verbose=2, n_jobs=-1)

In [ ]:
all_text_cv = np.mean(scores)
scores, all_text_cv

Так себе, попробуем перебрать вариации н-грамм, пока будем идти только по словам, вариацию с анализатором отдельных буквенных сочетаний сделаем дальше, также добавим ограничение на минимальное количество слов в корпусе для словаря, чтобы сильно сократить его размер за счет слов, встретившихся 1 раз.

In [ ]:
%%time
n_gram = [(1,1), (1, 2), (1, 3), (1,4)]
best_ngram = 0
best_r2 = 0
for n in n_gram:
    vect_desc_n = TfidfVectorizer(ngram_range=n, min_df=2)
    vect_name_n = TfidfVectorizer(ngram_range=n, min_df=2)
    vect_skills_n = TfidfVectorizer(ngram_range=n, min_df=2)
    
    X_train_desc = vect_desc_n.fit_transform(X_train['description'])
    X_train_name = vect_name_n.fit_transform(X_train['name'])
    X_train_skills = vect_skills_n.fit_transform(X_train['key_skills'])
    X_train_exp = scipy.sparse.csc_matrix(X_train['experience']).transpose()
    
    X_train_tfidf = hstack([X_train_desc, X_train_name, X_train_skills, X_train_exp])
    
    reg_test = SGDRegressor(max_iter=30000, learning_rate='adaptive',
                            penalty = 'l2', random_state=rng)
    reg_test.fit(X_train_tfidf, y_train)
    
    X_test_desc = vect_desc_n.transform(X_test['description'])
    X_test_name = vect_name_n.transform(X_test['name'])
    X_test_skills = vect_skills_n.transform(X_test['key_skills'])
    X_test_exp = scipy.sparse.csc_matrix(X_test['experience']).transpose()
    X_test_tfidf = hstack([X_test_desc, X_test_name, X_test_skills, X_test_exp])
    
    pred = reg_test.predict(X_test_tfidf)
    r2 = r2_score(y_test, pred)
    print(f'N_gramm: {n}, R2: {r2:.4f}')
    
    if r2 > best_r2: best_r2, best_ngram = r2, n 
    
    scores = cross_val_score(reg_test, X_train_tfidf, y_train,
                         scoring='r2', cv=5, n_jobs=-1)

    print(f'Cross-val scores: {scores}')
    print(f'Average cval score: {np.mean(scores):.4f}\n')

Использование юниграм+биграм дает прирост, попробуем оставить кодировку по словам с юни+биграммами для описаний, и использовать кодировку по буквенным сочетаниям для названия и кючевых навыков:

In [ ]:
# Функция для прогона разных вариаций векторизации:
# Добавение колонки с опытом дает прирост (тестил отдельно, убрал чтобы
# не раздвать тетрадку), но прилично увеличивает время на расчеты, пока мы
# перебираем варианты н-грамм и анализаторов, я уберу ее из расчетов

def vect_test(X_train, X_test, y_train, y_test, analyzer, n_words=None, n_chars=None):
    
    """Изменяем параметры анализа: перебираем анализатор
    (по слову, по буквам, по буквам с ограничением (токены только внутри слов),
    и вариации н-грамм для названия и ключевых навыков."""
    
    vect_desc_n = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=2)
    if analyzer == 'word':
        vect_name_n = TfidfVectorizer(analyzer=analyzer, ngram_range=n_words, min_df=2)
        vect_skills_n = TfidfVectorizer(analyzer=analyzer, ngram_range=n_words, min_df=2)
    else:
        vect_name_n = TfidfVectorizer(analyzer=analyzer, ngram_range=n_chars, min_df=2)
        vect_skills_n = TfidfVectorizer(analyzer=analyzer, ngram_range=n_chars, min_df=2)
        
    X_train_desc = vect_desc_n.fit_transform(X_train['description'])
    X_train_name = vect_name_n.fit_transform(X_train['name'])
    X_train_skills = vect_skills_n.fit_transform(X_train['key_skills'])
    X_train_tfidf = hstack([X_train_desc, X_train_name, X_train_skills])
    
    reg_test = SGDRegressor(max_iter=30000, learning_rate='adaptive',
                            penalty = 'l2', random_state=rng)
    reg_test.fit(X_train_tfidf, y_train)
    
    X_test_desc = vect_desc_n.transform(X_test['description'])
    X_test_name = vect_name_n.transform(X_test['name'])
    X_test_skills = vect_skills_n.transform(X_test['key_skills'])
    X_test_tfidf = hstack([X_test_desc, X_test_name, X_test_skills])

    pred = reg_test.predict(X_test_tfidf)
    r2 = r2_score(y_test, pred)
    
    scores = cross_val_score(reg_test, X_train_tfidf, y_train,
                         scoring='r2', cv=5, verbose=0, n_jobs=-1)
    

    return r2, scores

In [ ]:
%%time
n_words = [(1,1), (1,2), (1,3)]
n_chars = [(1,3), (1,5), (1,7), (1,9), (2,5), (2,9)]
analyzers = ['word', 'char', 'char_wb']
r2s = []
scoress = []
best_cv = 0
for analyzer in analyzers:
    if analyzer == 'word':
        for words in n_words:
            r2, scores = vect_test(X_train, X_test, y_train, y_test,
                             analyzer=analyzer, n_words=words, n_chars=None)
            print(analyzer, words, r2, scores, np.mean(scores))
            if np.mean(scores) > best_cv:
                best_cv, params = np.mean(scores), [analyzer, words]
                
    else: 
        for chars in n_chars:
            r2, scores = vect_test(X_train, X_test, y_train, y_test,
                             analyzer=analyzer, n_words=None, n_chars=chars)
            print(analyzer, chars, r2, scores, np.mean(scores))
            if np.mean(scores) > best_cv:
                best_cv, params = np.mean(scores), [analyzer, chars]

In [ ]:
best_cv, params, baseline_cv

Теперь соберем вместе лучшие параметры векторизации + данные об опыте (уменьшу количество фолдов на кросс валидации, иначе при 5, очень долго ждать схождения, но при 5 он показывал средний скор близкий к 49):

In [ ]:
vect_desc_n = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=2)
vect_name_n = TfidfVectorizer(analyzer='char_wb', ngram_range=(1,3), min_df=2)
vect_skills_n = TfidfVectorizer(analyzer='char_wb', ngram_range=(1,3), min_df=2)

X_train_desc = vect_desc_n.fit_transform(X_train['description'])
X_train_name = vect_name_n.fit_transform(X_train['name'])
X_train_skills = vect_skills_n.fit_transform(X_train['key_skills'])
X_train_exp = scipy.sparse.csc_matrix(X_train['experience']).transpose()

X_train_tfidf = hstack([X_train_desc, X_train_name, X_train_skills, X_train_exp])

reg_test = SGDRegressor(max_iter=40000, learning_rate='adaptive',
                        penalty = 'l2', random_state=rng)
reg_test.fit(X_train_tfidf, y_train)

X_test_desc = vect_desc_n.transform(X_test['description'])
X_test_name = vect_name_n.transform(X_test['name'])
X_test_skills = vect_skills_n.transform(X_test['key_skills'])
X_test_exp = scipy.sparse.csc_matrix(X_test['experience']).transpose()
X_test_tfidf = hstack([X_test_desc, X_test_name, X_test_skills, X_test_exp])

pred = reg_test.predict(X_test_tfidf)
r2 = r2_score(y_test, pred)
print(f'R2: {r2:.4f}')

scores = cross_val_score(reg_test, X_train_tfidf, y_train,
                     scoring='r2', cv=3, verbose=1, n_jobs=-1)

print(f'Cross-val scores: {scores}')
print(f'Average cval score: {np.mean(scores):.4f}\n')

Лучшим вариантом TF-IDF векторизации оказалось использование векторазации колонки описания по словам с юни и би-граммами, и колонок названий и ключевых навыков по буквенным сочетаниям (в пределах слова) с юни, би и три-граммами. Этот результат всего лишь на 3 (примерно) пункта R2 отстает от первого места с соревнования, где использовали BERT для получения эмбедингов, и хоть и простенькую, но все же нейронку для предсказания. Очень неплохо для простой по сути модели векторизации и довольно привычной регрессии.
___

#### Градиентный бустинг:
Прежде чем переходить к Word2Vec, попробуем скормить лучший вариант TF-IDF векторизации катбусту:

In [ ]:
# Вначале базовый вариант:
base_cat = CatBoostRegressor(eval_metric='R2')
base_cat.fit(X_train_tfidf, y_train, verbose=False, plot=True)

In [ ]:
pred = base_cat.predict(X_test_tfidf)

In [ ]:
score = r2_score(y_test, pred)
score

Хороший результат, но какое же долгое даже базовое обучение, подбор гиперпараметров может занять крайне много времени, попробуем снизить размерность данных, и скормить их библиотеке Optuna для оптимизации гиперпараметров:

In [ ]:
# Попробуем уменьшить размерность, чтобы ускорить обучение:
from sklearn.decomposition import TruncatedSVD

In [ ]:
# Перезададим базовые вектора (без колонки опыта)
vect_desc_n = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=2)
vect_name_n = TfidfVectorizer(analyzer='char_wb', ngram_range=(1,3), min_df=2)
vect_skills_n = TfidfVectorizer(analyzer='char_wb', ngram_range=(1,3), min_df=2)
X_train_desc = vect_desc_n.fit_transform(X_train['description'])
X_train_name = vect_name_n.fit_transform(X_train['name'])
X_train_skills = vect_skills_n.fit_transform(X_train['key_skills'])
X_train_tfidf = hstack([X_train_desc, X_train_name, X_train_skills])

X_test_desc = vect_desc_n.transform(X_test['description'])
X_test_name = vect_name_n.transform(X_test['name'])
X_test_skills = vect_skills_n.transform(X_test['key_skills'])
X_test_tfidf = hstack([X_test_desc, X_test_name, X_test_skills])

In [ ]:
svd = TruncatedSVD(n_components=1000, random_state=rng)
X_trans_train = svd.fit_transform(X_train_tfidf)
X_trans_test = svd.transform(X_test_tfidf)

Попробуем использовать базовые параметры, со сниженной размерностью:

In [ ]:
base_2_cat = CatBoostRegressor(eval_metric='R2')
base_2_cat.fit(X_trans_train, y_train, verbose=False, plot=True)

In [ ]:
pred = base_2_cat.predict(X_trans_test)
score = r2_score(y_test, pred)
score

Мда, теряется действительно много, теперь попробуем подобрать параметры:

In [ ]:
# Базовая функция для подбора параметров, пробовл перебирать очень много чего:
def objective(trial):
    
    params = {
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 1.0),
        "n_estimators": trial.suggest_int("n_estimators", 10, 5000)
#         'iterations':trial.suggest_int("iterations", 4000, 25000),
#         'od_wait':trial.suggest_int('od_wait', 500, 2300),
#         'learning_rate' : trial.suggest_uniform('learning_rate',0.001, 1),
#         'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
#         'subsample': trial.suggest_uniform('subsample',0,1),
#         'random_strength': trial.suggest_uniform('random_strength',10,50),
#         'depth': trial.suggest_int('depth',1, 15),
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,50),
#         'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
    }

    model = catboost.CatBoostRegressor(
        eval_metric='R2',
        random_state=rng,
        **params,
    )
    model.fit(X_trans_train, y_train)
    score = model.score(X_trans_test, y_test)
    
    return score

In [ ]:
# запускаем подбор параметров, вообще хочется поставить количиство попыток
# хотя бы 100, но даже 20 занимают долгое время, а катбуст не поддерживает расчеты
# на AMD GPU (в отличие например от торча).
# Этот подбор занял около 2 часов.

study_cat = optuna.create_study(direction="maximize")
study_cat.optimize(objective, n_trials=20, show_progress_bar=True)
study_cat.best_params

In [ ]:
best_model = CatBoostRegressor(**study_cat.best_params)
best_model.fit(trans, y_train, verbose=False, plot=True)

In [ ]:
pred = best_model.predict(trans_test)

In [ ]:
score = r2_score(y_test, pred)
score

In [ ]:
best_model.save_model('best_boost')

Мда, 2 часа обучения в пустую. Подбор параметров улучшил базовый результат (до 0.38). Снижение размерности сильно портит результат, а без него подбор парметров растягвается очень на долго. Ладно, как по мне, в данной задачи с такими векторами, стохастическая регрессия себя прекрасно показала, лучше СНАЧАЛА потратить время на дополниельные модификации признаков (параметры векторизации, и комбинации), чем тратить ночи (оптимизация реально могла идти всю ночь если оставить больше параметров) на катбуст, получая в итоге чуть худший результат.
Попробуем более продвинутый способ векторизации:

### Word2Vec 
В отличие от TF-IDF, Word2Vec выдает векторы одинаковой длинны для каждого слова, чаще всего используюся векторы длинны 300. Эти вектора обозначают некое значение каждого слова по 300 разным параметрам, эта можель уже улавливает семантику. Под капотом там простая 1 слойная нейронка, которая на вход получает двигающееся окно из нескольких слов. Обучается может 2 путями - Continious Bag of Words (CBOW), и Skip-Gram. В первом случае она учится предсказывать центральное слово в движущемся окне, на основе окружающих его слов. Во втором, учится предсказывать окружающие на основе данного центрального. Gensim построен на скип-граме (которая в целом используется чаще чем cbow).

In [ ]:
# Для обучения совместим все:
X_train_w2v = X_train['name'] + ' ' + X_train['description'] + ' ' + X_train['key_skills']

# список токенов для обучения
w2vec_tokens = [nltk.word_tokenize(element) for element in X_train_w2v]

# обучаем word2vec, гиперпараметры подбирал раньше, это одни из оптимальных для этой задачи

w2v = Word2Vec(w2vec_tokens, sg=1, hs=1,
                     vector_size=300, window=9, min_count=1, workers=16, epochs=10)

In [ ]:
w2v.wv.most_similar(['грузчик'])

Мы получили эмбеддинги для всех слов, но нам нужны эмбеддинги всего описания, попробуем взять взвешенный средний эмбеддинг по всем словам к каждом описании, где весом слова будет его TF-IDF.

In [ ]:
# Берем веса из базового tfidf высчитанного ранее, поскольку word2vec считает только
# слова, всякие биграммы и токенизация по буквенным сочетаниям нам тут не нужна,
# поэтоум берем базовый:
weights = dict(zip(baseline_tfidf.get_feature_names_out(), baseline_tfidf.idf_))


In [ ]:
# Функция расчитывающая эмбеддинг для всего описания + название + навыки

def w2v_vects(texts):
    w2v_features = []
    words_count = 0
    for element in texts:
        vect = np.zeros(300)
        tokens = nltk.word_tokenize(element)
        for token in tokens:
            words_count += 1
            if (token in w2v.wv.key_to_index.keys() and (token in weights)):
                word_vector = w2v.wv[token]
                vect = vect + word_vector * weights[token]
        
        if words_count == 0:
            words_count = 1
        w2v_features.append(vect/words_count)
        words_count = 0
                
    return w2v_features

In [ ]:
features_w2v_train = w2v_vects(X_train_w2v)

In [ ]:
X_test_w2v = X_test['name'] + ' ' + X_test['description'] + ' ' + X_test['key_skills']
features_w2v_test = w2v_vects(X_test_w2v)

In [ ]:
features_w2v_train = pd.DataFrame(features_w2v_train)
features_w2v_test = pd.DataFrame(features_w2v_test)

Попробуем использовать базовую модель:

In [ ]:
reg_test = SGDRegressor(max_iter=30000, learning_rate='adaptive',
                            penalty = 'l2', random_state=rng)
    
reg_test.fit(features_w2v_train, y_train)


scores = cross_val_score(reg_test, features_w2v_train, y_train,
                     scoring='r2', cv=5, verbose=2, n_jobs=-1)

In [ ]:
scores

Здесб уже регрессия начинает выдавать дичь, вероятно можно поиграться с нормированием признаков, и зафайнтюнить регрессию так чтобы он дала нормальный результат, НО при этом катбуст спрвляется сразу, и учится во много раз быстрее чем с TF-IDF эмбеддингами.

In [ ]:
base_cat = CatBoostRegressor(eval_metric='R2', learning_rate=0.06, n_estimators=3000)
base_cat.fit(features_w2v_train, y_train, silent=True, plot=True)

In [ ]:
pred = base_cat.predict(features_w2v_test)

In [ ]:
score = r2_score(y_test, pred)
score

In [ ]:
%%time
result =  cross_val_score(base_cat, features_w2v_train, y_train, cv=5)
print(result.mean())

In [ ]:
# Базовая функция для подбора параметров, пробовл перебирать очень много чего:
def objective(trial):
    
    params = {
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.03, 1.0),
        "n_estimators": trial.suggest_int("n_estimators", 10, 5000)

    }

    model = catboost.CatBoostRegressor(
        eval_metric='R2',
        random_state=rng,
        **params,
    )
    model.fit(features_w2v_train, y_train)
    score = model.score(features_w2v_test, y_test)
    
    return score

In [ ]:
study_cat = optuna.create_study(direction="maximize")
study_cat.optimize(objective, n_trials=20, show_progress_bar=True)
study_cat.best_params

In [ ]:
best_model = CatBoostRegressor(**study_cat.best_params)
best_model.fit(features_w2v_train, y_train, verbose=False, plot=True)

In [ ]:
pred = best_model.predict(features_w2v_test)

In [ ]:
score = r2_score(y_test, pred)
score

#### Заключение:
Лучший результат в 0.48, на кросс валидации и 0.49 на тесте от TF-IDF склеенного с опытом работы, и скормленного стохастической линейной регрессии с l-2 регуляризатором. Бейзлайн давал примерно 0.445 пункта везде. Подозреваю что можно подкрутить Word2Vec, поскольку основная егго проблема тут - выборка для обучения маловата. Для хорошего word2vec нужен огромный корпус. Но все равно, в этой задаче TF-IDF + SGDRegressor показывают себя очень хорошо, близко к существенно более тяжеловесным моделям. Подозреваю что это из-за специфики задачи, тут не важен тон/настроение, еще какие либо характеристки текста, которые могут уловить трансформеры, да и прогноз 1й цифры очень хорошо выполняется регрессией.